In [70]:
%serialconnect


Connecting to --port=/dev/ttyUSB0 --baud=115200 
Traceback (most recent call last):
  File "main.py", line 53, in <module>
  File "UDPblackviewphone.py", line 19, in connectActivePhone
KeyboardInterrupt: 
MicroPython v1.13-254-g1719459c2 on 2020-12-14; ESP32 module with ESP32
Type "help()" for more information.
>>> 
>>> 
MicroPython v1.13-254-g1719459c2 on 2020-12-14; ESP32 module with ESP32
Type "help()" for more information.
>>> 
>>> 
raw REPL; CTRL-B to exit
>Ready.


In [2]:
%sendtofile --mkdir --source /home/timbo/extrepos/py-esp32-ulp/esp32_ulp  esp32_ulp

Sent 283 lines (9521 bytes) to esp32_ulp/assemble.py.


In [13]:
from esp32_ulp.__main__ import src_to_binary

source = """\
dataiorec:  .long 0x00000000
timepin27:  .long 0x00000000
countpin27: .long 0x00000000
timepin33:  .long 0x00000000
countpin33: .long 0x00000000
entry:
            move r3, dataiorec
            move r1, 0x0000     // pin27 counter
            move r2, 0x0000     // pin33 counter
mainloop:
            reg_rd 0x3ff48424, 16, 17   // read IO register into r0
            st   r0, r3, 0      // dataiorec[0]=r0
            rsh  r0, r0, 14     // access pin 27
            and  r0, r0, 0x0001

            jump p27eq0, eq
            add  r1, r1, 0x0001
            jump p27eq0end
p27eq0:
            add  r1, r1, 0x0000 // update the zero flag
            jump p27eq0end, eq
            st   r1, r3, 4      // dataiorec[1]=r1
            move r1, 0x0000     // r1=0

            ld   r0, r3, 8      // r0=dataiorec[2] 
            add  r0, r0, 1      // r0++
            st   r0, r3, 8      // dataiorec[2]=r0

p27eq0end:

            reg_rd 0x3ff48424, 16, 17   // read IO register into r0
            st   r0, r3, 0      // dataiorec[0]=r0
            rsh  r0, r0, 5      // access pin 33
            and  r0, r0, 0x0001

            jump p33eq0, eq
            add  r2, r2, 0x0001
            jump p33eq0end
p33eq0:
            add  r2, r2, 0x0000 // update the zero flag
            jump p33eq0end, eq
            st   r2, r3, 12      // dataiorec[3]=r2
            move r2, 0x0000     // r2=0

            ld   r0, r3, 16     // r0=dataiorec[4] 
            add  r0, r0, 1      // r0++
            st   r0, r3, 16     // dataiorec[4]=r0

p33eq0end:
            jump mainloop
            halt
"""
binary = src_to_binary(source)
fout = open("measure_pwm.ulp", "wb")
fout.write(binary)
fout.close()

0000 dataiorec
0001 timepin27
0002 countpin27
0003 timepin33
0004 countpin33
0005 entry
0008 mainloop
0015 p27eq0
0022 p27eq0end
0029 p33eq0
0036 p33eq0end


In [9]:
# Now we can load this assembled code into the ULP and set it running
# Don't forget to initialize the pins
%ls

Listing directory '/'.
     5060    BNO055_class.py
     1123    UDPblackviewphone.py
      139    boot.py
     1350    calibfileZ.txt
      169    config.txt
             esp32_ulp/
     3376    main.py
      164    measure_pwm.ulp


In [68]:
%sendtofile --source BNO055_class.py

Sent 142 lines (5506 bytes) to BNO055_class.py.


In [3]:
%sendtofile --source UDPblackviewphone.py

Sent 42 lines (1123 bytes) to UDPblackviewphone.py.


In [29]:
%sendtofile config.txt

pinled       13
deviceletter @
connection0  trilbynet,flying7tiara,192.168.43.1,9042
dev1         1,rx25,tx26,Z
dev2         2,rx36,tx4,Y
pitchServo   33
rollServo    27

Sent 7 lines (169 bytes) to config.txt.


In [ ]:
# scl=25(A1) sda=26(A0)
# scl=tx sda=rx

# scl=36(A4) sda=4(A5)
# scl=tx sda=rx



In [5]:
%sendtofile --source HangDualBNO055.py main.py

Sent 75 lines (2297 bytes) to main.py.


In [71]:
#%sendtofile main.py
import socket, urandom, machine
import time, socket, urandom, esp32
from BNO055_class import BNO055
from UDPblackviewphone import connectActivePhone

fconfig = dict(x.strip().split(None, 1)  for x in open("config.txt"))
pinled = machine.Pin(int(fconfig["pinled"]), machine.Pin.OUT)
pinled.value(1)

def MakeBNO055(dev):
    d = dev.split(",")
    u = machine.UART(int(d[0]), baudrate=115200, rx=int(d[1][2:]), tx=int(d[2][2:]))
    for i in range(5):
        try:
            print(u, d[3])
            return BNO055(u, d[3])
        except OSError as e:
            print(e)
        print("fail", i, d)
        time.sleep(0.5)
        
def readulpdata(r):
    return machine.mem32[0x50000000+r]&0xFFFF


    
dev1 = MakeBNO055(fconfig["dev1"])
dev2 = MakeBNO055(fconfig["dev2"])


pitchServo = machine.Pin(int(fconfig["pitchServo"]), machine.Pin.IN)
rollServo = machine.Pin(int(fconfig["rollServo"]), machine.Pin.IN)
pitchServoLast = 0
rollServoLast = 0

ulp = esp32.ULP()
ulp.load_binary(0, open("measure_pwm.ulp", "rb").read())
ulp.set_wakeup_period(0, 1000000)
ulp.run(5*4)

deviceletter = fconfig["deviceletter"]
c = fconfig["connection0"].split(",")
hotspots = { c[0].encode(): (c[1].encode(), c[2], int(c[3])) }
androidipnumber = c[2]
portnumber = int(c[3])
print("deviceletter", deviceletter)

while not connectActivePhone(pinled, hotspots):
    for j in range(3):
        for i in range(7):
            pinled.value(i%2)
            time.sleep_ms(100)
        time.sleep_ms(800)

prevflushstamp = 0
nextledonstamp = 0

ebs = bytearray("Et00000000m00d0000\n")
mebs = memoryview(ebs)

while True:
    try:
        ss = socket.socket()
        ss.settimeout(1)
        print(ss)
        ss.connect(socket.getaddrinfo(androidipnumber, portnumber)[0][-1])
        s = ss.makefile('rwb', 0)
        print(s.readline())
        s.write(b"%c%c%c%c"%(deviceletter,deviceletter,deviceletter,deviceletter))
        dwrite = s.write
        while True:
            tstamp = time.ticks_ms()
            if tstamp > nextledonstamp:
                pv = (0 if pinled.value() else 1)
                pinled.value(pv)
                nextledonstamp = tstamp + 1000//(10 if pv else 1)
            if dev1 is not None:
                b1 = dev1.readhexlifyBNO055(10)
                if b1 is not None:
                    dwrite(b1)
            if dev2 is not None:
                b2 = dev2.readhexlifyBNO055(10)
                if b2 is not None:
                    dwrite(b2)
                    
            pitchServoNow = readulpdata(12)
            rollServoNow = readulpdata(4)
            if pitchServoNow != pitchServoLast:
                if pitchServoNow <1500:
                    mebs[2:10] = b"%08X" % tstamp
                    mebs[12] = ord('1')
                    mebs[14:18] = b"%04X"% pitchServoNow
                    dwrite(ebs)
                    #print('Pitch: ', ebs)
                pitchServoLast = pitchServoNow
                
            if rollServoNow != rollServoLast:
                if rollServoNow <1500:
                    mebs[2:10] = b"%08X" % tstamp
                    mebs[12] = ord('2')
                    mebs[14:18] = b"%04X"% rollServoNow
                    dwrite(ebs)
                    #print('Roll: ', ebs)
                rollServoLast = rollServoNow

                
    except OSError as e:
        print("OSError", e)
        time.sleep(2)

UART(1, baudrate=115201, bits=8, parity=None, stop=1, tx=26, rx=25, rts=-1, cts=-1, txbuf=256, rxbuf=256, timeout=0, timeout_char=1) Z
0000B2BC 0000000000000000000000000000000000000000E001

b'\xee\x01'
Temp29
UART(2, baudrate=115201, bits=8, parity=None, stop=1, tx=4, rx=36, rts=-1, cts=-1, txbuf=256, rxbuf=256, timeout=0, timeout_char=1) Y
0000950C FBFF40000C007DFF42007001FFFF0000FFFFE8030E03

b'\xee\x01'
Temp27
deviceletter @
{b'trilbynet': (b'flying7tiara', '192.168.43.1', 9042)}
(b'trilbynet', b'.\x0e=\x98\xcf\x7f', 1, -40, 3, False)
<socket>
b'Hello from Android thread 3\n'
None bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')
None bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')
36 bytearray(b'\xbb"\x00\x00\x02\x00\xff\xffa\x07\xed\xff8\xfao%W\xd83\xe8\x91\x17\xfd

36 bytearray(b'\xbb"\x02\x00\xfe\xff\xff\xff\xe6\x0cn\x00\xb1\xf9A&\xf7\xcfC\x0b\xe9\xf1\x05\x00\xfe\xff\x04\x00u\x00E\xfcH\xff\x1b|')
36 bytearray(b'\xbb"\xfe\xff\xfd\xff\xff\xffa\x07\xed\xff8\xfao%W\xd83\xe8\x91\x17\xff\xff\x05\x00\x03\x00\xec\xff-\xfc\xd5\xff\x1d\xff')
36 bytearray(b'\xbb"\x02\x00\xff\xff\x01\x00\xe6\x0cn\x00\xb1\xf9A&\xf7\xcfC\x0b\xe9\xf1\x04\x00\xff\xff\x07\x00u\x00E\xfcH\xff\x1b|')
36 bytearray(b'\xbb"\x03\x00\x01\x00\x01\x00a\x07\xed\xff8\xfao%W\xd83\xe8\x91\x17\xfc\xff\x07\x00\x01\x00\xec\xff-\xfc\xd5\xff\x1d\xff')
36 bytearray(b'\xbb"\x03\x00\x00\x00\x00\x00\xe6\x0cn\x00\xb1\xf9A&\xf7\xcfC\x0b\xe9\xf1\x05\x00\xfe\xff\x06\x00u\x00E\xfcH\xff\x1b|')
36 bytearray(b'\xbb"\x00\x00\xfe\xff\xff\xffa\x07\xed\xff8\xfao%W\xd83\xe8\x91\x17\xfd\xff\x06\x00\x00\x00\xec\xff-\xfc\xd5\xff\x1d\xff')
36 bytearray(b'\xbb"\x01\x00\x02\x00\x00\x00\xe6\x0cn\x00\xb1\xf9A&\xf7\xcfC\x0b\xe9\xf1\x06\x00\xfe\xff\x07\x00u\x00E\xfcH\xff\x1b|')
36 bytearray(b'\xbb"\x01\x00\x01\x00\xff\xffa\

36 bytearray(b'\xbb"\x04\x00\x02\x00\x01\x00a\x07\xed\xff8\xfao%W\xd83\xe8\x91\x17\xfd\xff\x06\x00\x03\x00\xec\xff-\xfc\xd5\xff\x1d\xff')
36 bytearray(b'\xbb"\xfe\xff\xfe\xff\xff\xff\xe6\x0cn\x00\xb1\xf9A&\xf7\xcfC\x0b\xe9\xf1\x05\x00\x00\x00\x07\x00u\x00E\xfcH\xff\x1b<')
36 bytearray(b'\xbb"\x01\x00\xfe\xff\x02\x00a\x07\xed\xff8\xfao%W\xd83\xe8\x91\x17\xfa\xff\t\x00\xfb\xff\xec\xff-\xfc\xd5\xff\x1d\xff')
36 bytearray(b'\xbb"\xff\xff\xff\xff\x00\x00\xe6\x0cn\x00\xb1\xf9A&\xf7\xcfC\x0b\xe9\xf1\x06\x00\xfd\xff\x06\x00u\x00E\xfcH\xff\x1b<')
36 bytearray(b'\xbb"\x00\x00\x00\x00\x01\x00a\x07\xed\xff8\xfao%W\xd83\xe8\x91\x17\xfe\xff\xfd\xff\x03\x00\xec\xff-\xfc\xd5\xff\x1d\xff')
36 bytearray(b'\xbb"\xf5\xff\xff\xff\xfb\xff\xe6\x0cn\x00\xb1\xf9A&\xf7\xcfC\x0b\xe9\xf1\x01\x00\x17\x00\t\x00u\x00E\xfcH\xff\x1b<')
36 bytearray(b'\xbb"\x06\x00\x02\x00\xfb\xffa\x07\xed\xff8\xfao%W\xd83\xe8\x91\x17\xfb\xff\x03\x00\x00\x00\xec\xff-\xfc\xd5\xff\x1d\xff')
36 bytearray(b'\xbb"\x04\x00\x01\x00\xff\xff\xe

36 bytearray(b'\xbb"`\t\x8a\xfd/\x00\xba\r\xd0\xff#\xf9E#\xbf\xce\xc5\x11~\xf5\xce\x00\xaf\x00\x95\xff\xcc\xffg\xfc\xb4\xfe\x1b>')
36 bytearray(b'\xbb"5\xf8\xae\xfe0\xfb\x08\x06\xd7\x00\xfa\xf8\xf0\x1fy\xd8\xba\xdcj\x10~\xff&\x00\xa1\xff\xe4\x00\x8f\xfc\x95\xfe\x1d\xff')
36 bytearray(b'\xbb"\x1f\t[\xfeU\x00\xb7\r\xcc\xffQ\xf9\x8c$\x95\xcf[\x11L\xf5\x83\x00{\x00\xc5\xff\xc9\xffX\xfc\xe2\xfe\x1b>')
36 bytearray(b'\xbb"\x0c\xf8g\xfe\xbc\xfd\xfc\x05\xe1\x00\xd0\xf8\x0e\x1f \xd8\xf6\xdb\x96\x0f\xf7\xffS\x00\xec\xff\xee\x00\xa3\xfcn\xfe\x1d\xff')
2 bytearray(b'\xee\x07\x1f\t[\xfeU\x00\xb7\r\xcc\xffQ\xf9\x8c$\x95\xcf[\x11L\xf5\x83\x00{\x00\xc5\xff\xc9\xffX\xfc\xe2\xfe\x1b>')
36 bytearray(b'\xbb"\xcb\xf7O\xfe>\xfd\xf2\x05\xe8\x00\xa4\xf8\x1d\x1e\xc3\xd7:\xdb\xc4\x0e\xb7\xff-\x00\xf2\xff\xf5\x00\xb8\xfcF\xfe\x1d\xff')
36 bytearray(b'\xbb"_\t\xd1\xfe\xf4\xfe\xa3\r\xdd\xff\xbb\xf9>\'\x92\xd1\x16\x10\x19\xf4H\xffl\xff~\xff\xdb\xff<\xfcQ\xff\x1b>')
36 bytearray(b'\xbb"\xec\xf6w\xfd\xfa\xf9\xcf\x05\

36 bytearray(b'\xbb"\x1d\xfe\x11\x01@\x01Y\x03\xc7\xff\xd6\xf4r\x01\xd8\x1c\x189o\xfe\x82\xff3\x00\xb7\xff\xc3\xff\xea\xff.\xfc\x1d\xbf')
36 bytearray(b'\xbb"C\xf1\x05\x06\x87\x01D\x01\xe6\x01\xa1\xfd\xbf9\xab\xea\xdc\xf1\xb0\xf5~\x02\xae\xfe \x00\xf0\x01\xf8\xfd\x9b\x02\x1b?')
36 bytearray(b'\xbb"\x9c\xfdF\xff\xfe\x01g\x03\xc7\xff\xc5\xf4\xb5\x01C\x1d\xe28\xf0\xfe\xeb\xff\x00\x00\xbc\x00\xc3\xff\xfc\xff.\xfc\x1d\xbf')
36 bytearray(b'\xbb"=\xf5\xcb\t\x19\xfei\x01\xc5\x01\x80\xfdj9Q\xe9\x9d\xf3\x87\xf4\xe1\xff\x02\xff\xd4\xff\xd1\x01\xd5\xfd\x94\x02\x1b?')
36 bytearray(b'\xbb"\xb6\xfd\x1c\xff \x05|\x03\xc3\xff7\x0b\n\x02\xee\x1d\x878B\xff\x18\x008\x00\xe2\xff\xbf\xff\t\x00.\xfc\x1d\xbf')
36 bytearray(b'\xbb"\x1b\xf9\xc9\x05c\xfc\xb0\x01\xb2\x01_\xfd\xb28\xe1\xe7O\xf5b\xf2\x02\x00\x84\xff\xb9\xff\xbf\x01\xb7\xfd\x87\x02\x1b?')
36 bytearray(b'\xbb"G\xfe>\x00l\x04\xa1\x03\xc2\xff&\x0b\\\x02\r\x1f\xea7\xb8\xff\x91\x00\xe9\xff+\x00\xbd\xff\x1b\x00/\xfc\x1d\xff')
36 bytearray(b'\xbb"\xf2\xfe\

...

36 bytearray(b'\xbb"k\x04\xaa\x00E\x03\xfd\x13\xf7\xff\x05\xf5$\x01\x08\xea\x10<\xe3\xfd\x80\xff\xda\x00B\xff\xf6\xff\xb8\xff/\xfc\x1d\xff')
36 bytearray(b'\xbb"\x90\xff\t\x007\xff\xd6\x13N\x00\xdb\x00(;\x17\x084\x00\x0b\x17\xd5\xff\x8e\x00H\x00S\x00\xe7\x00\xb5\x03\x1b?')
36 bytearray(b'\xbb"\x87\x03\xb4\xffs\x02\x10\x14\xf4\xff"\xf5\x88\x01\xad\xea?<\xf1\xfc\xdb\xff\x01\x01\xbe\xff\xf4\xff\x99\xff1\xfc\x1d\xff')
36 bytearray(b'\xbb"r\x02\x84\x01\x81\xfd\xe1\x13F\x00\xe4\x00A;Q\x08\x88\x00\xb5\x16\x08\x00\xcd\x00\t\x00K\x00\xf1\x00\xb3\x03\x1b?')
36 bytearray(b'\xbb"\xbb\x02w\xff\xbe\x02\x1d\x14\xf0\xff0\xf5\xce\x01\x1e\xeb_<\x81\xfc\xff\xff\xcb\x00\xa8\xff\xef\xff\x89\xff3\xfc\x1d\xff')
36 bytearray(b'\xbb"\\\x00\x13\x02\xfb\xfc\xec\x13/\x00\xee\x00b;W\x08]\x01S\x16\xae\xff\xd3\x00\xd5\xff2\x00\xfb\x00\xb2\x03\x1b?')
36 bytearray(b'\xbb"\x16\x02a\xffx\x02)\x14\xec\xff;\xf5\r\x02\x87\xeb|<0\xfc\x0b\x00\xdb\x00\x88\xff\xea\xff~\xff5\xfc\x1d\xff')
36 bytearray(b'\xbb"\n\xfe\xbc\x00J\xfd

36 bytearray(b'\xbb"M\xf6N\x00]\xfc\xf3\x15\x9d\x00\xd3\x08f\x07\x9f\x04\x00\xc4\x81\xebk\x00\xc5\x00+\xff\xa7\x00]\x02\x0f\xfd\x1c\xbf')
36 bytearray(b'\x1b\x7f\xbb"\xd6\x12\xe3\x0b\xd4\x0b \x15\xfc\xfd#\x04s3\xb4\x19>\x1a\xfb\tU\x00v\xfd\x9d\x00\xf5\xfd\xf6\x02N\x01')
36 bytearray(b'\xbb"\x94\xf2\xe4\x00\xd4\xfd\xeb\x15\x97\x00\x93\x08\x87\x07\xd2\x04\xc6\xc4s\xe9g\x00b\x00\xf4\xfe\xa1\x00\x90\x02:\xfd\x1c\xbf')
36 bytearray(b'\x1b\x7f\xbb"\xb6\x15x\t\x05\x0c\xc5\x14A\xfe\xf9\x04S/\xbd\x1e\xdc\x1b\xa6\x0b\x18\xff\t\xfd\xf6\x005\xfeS\x03\x9c\x00')
36 bytearray(b'\xbb"*\xee \x03\xa3\x01\r\t\xa2\x00\x17\x08Z\x08\x87\x04y\xc6\x9c\xe5V\x00S\x00\xc6\xfe\xac\x00\xe9\x02\x9b\xfd\x1c\xbf')
36 bytearray(b'\x1b\x7f\xbb"s\x1d\x0e\x02\xeb\x06\xca\x14e\xfe\x8d\x05],\xa4"{\x1c\xce\n2\x006\xfd\xff\xffW\xfes\x03\x11\x00')
36 bytearray(b'\xbb"_\xed\x90\x04S\x02\x03\t\xb8\x00\xb9\x07g\t\x04\x04\xf1\xc7\xdd\xe2M\x00\x02\x00\xe0\xfe\xc4\x00 \x03\xed\xfd\x1c\xbf')
36 bytearray(b'\x1b\x7f\xbb"\x82\'1\xff\x

36 bytearray(b'\xbb"\xd9\xfe\xf5\xfb\xc8\x03\xe9\x07T\xffy\n\xd0\x08\x978w\x1c\xa7\xfdJ\x00~\xff5\x00I\xff\xce\x00T\xfc\x1b\xbf')
36 bytearray(b'\xbb"1\xfc\xd7\xffL\x00\xa0\x08\x98\x01\x14\x03q\x14Y\xfa\x8b\xe3\xbe\xca,\x00\x9c\xff\x7f\xff\xa6\x01\x9e\x02A\x02\x1c?')
36 bytearray(b'\xbb"c\x05\xfb\xf9\xe5\x03\x0e\x083\xff\x85\n\xde\x08\x169I\x1bD\xfcT\x00\xa4\xff\x7f\xff\'\xff\xc1\x00X\xfc\x1b\xbf')
36 bytearray(b'\xbb"\xcb\xfe\xbb\x00\xcd\xff\x97\x08\x96\x01\x02\x03\xc8\x14m\xfa\x07\xe4\x9c\xca\t\x00\xba\xff\xf2\xff\xa4\x01\x93\x02O\x02\x1c?')
36 bytearray(b'\xbb"[\t\x08\xfbw\x04+\x08\x1d\xff\xa9\n\xfb\x07\x8a9a\x1a\xeb\xfa>\x00J\x00\'\xff\x10\xff\x9b\x00W\xfc\x1b\xbf')
36 bytearray(b'\xbb"\xb8\xffB\x00\xb2\xff\x98\x08\x94\x01\xff\x02\xc6\x14|\xfa\x1c\xe4\x8f\xca\xcc\xff\xdd\xffL\x00\xa2\x01\x92\x02Q\x02\x1c?')
36 bytearray(b'\xbb"h\x04y\xfe\x04\x00G\x08\x0c\xff\xd8\n\xa3\x06\xf59\x89\x19\xa0\xf9\x00\x00\xd1\x00\x18\x00\xfe\xfej\x00T\xfc\x1b\xbf')
36 bytearray(b'\xbb"{\xff \xff\xc5\xff

36 bytearray(b'\xbb"\n\x01\x8f\xffq\x00\x0e\x08B\x01_\x03R\x18\x9f\x00\xb9\xe1"\xcd\x12\x00\t\x009\x00Q\x01\xe8\x02\x1d\x02\x1c\xbf')
36 bytearray(b'\x1b\xbf\xbb"\x86\xfd\xb1\xff\xc4\xff\xc5\x070\xff\x16\ne\x0c\x827L\x1d;\xfe\x07\x00F\xff\xf9\xff$\xff0\x01w\xfc')
36 bytearray(b'\xbb"`\x00g\xff3\x00\x07\x08D\x01b\x03z\x18\xa9\x00\x9d\xe1F\xcd\x1e\x00W\x00K\x00T\x01\xe9\x02\x1a\x02\x1c\xbf')
36 bytearray(b'\x1b\xbf\xbb"$\xfd\x04\x00\xcb\xff\xc3\x070\xff\t\n\xce\x0cf7W\x1do\xfe5\x00"\xff\x0e\x00#\xff=\x01|\xfc')
36 bytearray(b'\xbb"\x12\xff\x8d\xff\x83\x00\x07\x08G\x01`\x03}\x18\x85\x00\xa8\xe1A\xcd+\x00o\x00u\x00W\x01\xe7\x02\x1b\x02\x1c\xbf')
36 bytearray(b'\x1b\xbf\xbb"\x02\xfd\xf4\xff\x8d\xff\xbb\x07/\xff\xfa\tD\r47\x83\x1d\xad\xfe\t\x00\x06\xff\xc7\xff"\xffK\x01\x81\xfc')
36 bytearray(b'\xbb"I\xfd.\xff\x8f\x01\x03\x08P\x01O\x03\xb2\x18\x12\x00\x0e\xe2\x1d\xcd\x89\xff\x08\x00\x1d\x00`\x01\xda\x02\'\x02\x1c\xbf')
4 bytearray(b'\x1b\xbf\xee\x07\x02\xfd\xf4\xff\x8d\xff\xbb\x07/\xff\xfa\t

**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)


**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)



In [ ]:
#SCRATCH PAD STUFF FROM HERE ON:

In [33]:
import machine, time, esp32

p27 = machine.Pin(27, machine.Pin.IN)
p33 = machine.Pin(33, machine.Pin.IN)
print(p27.value(), p33.value())

ulp = esp32.ULP()
ulp.load_binary(0, open("measure_pwm.ulp", "rb").read())
ulp.set_wakeup_period(0, 1000000)
ulp.run(5*4)
def readulpdata(r):
    return machine.mem32[0x50000000+r]&0xFFFF

1 1


In [17]:
#w = machine.PWM(machine.Pin(5))
#w.freq(50)
#w.duty(51)

import time
while True:
    print(print(readulpdata(4), readulpdata(8), readulpdata(12), readulpdata(16)))
    time.sleep(0.2)

0 0 77 257
None
0 0 81 267
None
0 0 102 277
None
0 0 138 287
None
0 0 178 297
None
0 0 216 307
None
0 0 250 317
None
0 0 269 328
None
0 0 271 338
None
0 0 254 348
None
0 0 225 358
None
0 0 186 368
None
0 0 146 378
None
0 0 110 388
None
0 0 85 398
None
0 0 76 408
None
0 0 83 418
None
0 0 108 428
None
0 0 140 438
None
0 0 182 448
None
0 0 222 459
None
0 0 255 469
None
0 0 272 479
None
0 0 269 489
None
0 0 251 499
None
0 0 221 509
None
0 0 182 519
None
0 0 142 529
None
0 0 108 539
None
0 0 83 549
None
0 0 74 559
None
0 0 86 569
None
0 0 109 579
None
0 0 149 590
None
0 0 191 600
None
0 0 228 610
None
0 0 256 620
None
0 0 272 630
None
0 0 269 640
None
0 0 250 650
None
0 0 217 660
None
0 0 177 670
None
0 0 138 680
None
0 0 106 690
None
0 0 81 701
None
0 0 77 711
None
0 0 87 721
None
0 0 116 731
None
0 0 153 741
None
0 0 195 751
None
0 0 232 761
None
0 0 258 771
None
0 0 272 781
None
0 0 267 791
None
0 0 247 801
None
0 0 215 811
None
0 0 173 821
None
0 0 133 831
None
0 0 99 842
None
0 0 79 85

None
0 0 201 4953
None
0 0 239 4963
None
0 0 263 4973
None
0 0 272 4984
None
0 0 263 4994
None
0 0 239 5004
None
0 0 202 5014
None
0 0 162 5024
None
0 0 123 5034
None
0 0 94 5044
None
0 0 77 5054
None
0 0 79 5064
None
0 0 96 5074
None
0 0 128 5084
None
0 0 164 5094
None
0 0 211 5105
None
0 0 243 5115
None
0 0 265 5125
None
0 0 271 5135
None
0 0 261 5145
None
0 0 234 5155
None
0 0 199 5165
None
0 0 158 5175
None
0 0 121 5185
None
0 0 92 5195
None
0 0 77 5205
None
0 0 79 5215
None
0 0 99 5225
None
0 0 133 5236
None
0 0 173 5246
None
0 0 215 5256
None
0 0 248 5266
None
0 0 267 5276
None
0 0 272 5286
None
0 0 259 5296
None
0 0 232 5306
None
0 0 195 5316
None
0 0 154 5326
None
0 0 116 5336
None
0 0 90 5346
None
0 0 77 5356
None
0 0 81 5367
None
0 0 103 5377
None
0 0 138 5387
None
0 0 177 5397
None
0 0 217 5407
None
0 0 250 5417
None
0 0 270 5427
None
0 0 272 5437
None
0 0 257 5447
None
0 0 227 5457
None
0 0 191 5467
None
0 0 149 5477
None
0 0 114 5487
None
0 0 86 5498
None
0 0 77 5508
None


Traceback (most recent call last):
  File "<stdin>", line 8, in <module>
KeyboardInterrupt: 


In [2]:
# Code for making servo PWM
#%serialconnect
#%sendtofile main.py
import machine
import time

p = machine.PWM(machine.Pin(12), freq=100, duty=120)
p.freq(100)
p.duty(120)
time.sleep_ms(500)

while 1:
    for i in range(120, 180):
        p.duty(i)
        time.sleep_ms(20)
    for i in range(180, 120, -1):
        p.duty(i)
        time.sleep_ms(20)


...........

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 15, in <module>
KeyboardInterrupt: 


In [13]:
import machine
import time
p12 = machine.Pin(12, machine.Pin.IN)
p13 = machine.Pin(13, machine.Pin.IN)
while True:
    print(machine.time_pulse_us(p12, 1))
    #print(p13.value())
    print(time.ticks_ms())
    #time.sleep(1)



***Connection broken [Input/output error]
You may need to reconnect
Closing serial Serial<id=0x7fa4115ad790, open=True>(port='/dev/ttyUSB1', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
No serial connected
  %serialconnect to connect
  %esptool to flash the device
  %lsmagic to list commands

In [47]:
import machine
p12 = machine.Pin(12, machine.Pin.IN, machine.Pin.PULL_DOWN)

In [82]:
print("".join("%d" % p.value()  for i in range(80)))

00000100000100000010000001000000100000100000010000001000000100000000000010000001


In [79]:
help(p.irq)

object <bound_method> is of type bound_method


In [48]:
import time
p12up = -1
p12dd = -1
def p12irq(p):
    global p12up, p12dd
    if p.value() == 1:
        p12up = time.ticks_us()
    else:
        p12dd = time.ticks_us() - p12up
        p12up = -1
p12.irq(handler=p12irq, trigger=machine.Pin.IRQ_FALLING | machine.Pin.IRQ_RISING)
       

In [50]:
for i in range(250):
    while p12dd == -1:
        pass
    print(p12dd, ',',end=" ")
    p12dd = -1
    #if (i % 15) == 14:
        #print()


499015477 , 744 , 578 , 784 , 779 , 720 , 805 , 622 , 812 , 823 , 761 , 819 , 560 , 854 , 853 , 815 , 867 , 879 , 887 , 888 , 865 , 908 , 621 , 929 , 934 , 877 , 951 , 948 , 962 , 968 , 899 , 976 , 816 , 817 , 1023 , 958 , 1021 , 1046 , 1043 , 1055 , 1016 , 1093 , 1086 , 807 , 1105 , 1062 , 1133 , 1105 , 1144 , 1169 , 1116 , 1186 , 1190 , 919 , 1199 , 1148 , 1219 , 1242 , 1242 , 1439 , 1230 , 1274 , 1294 , 999 , 1318 , 1253 , 1336 , 1340 , 1351 , 1355 , 1329 , 1377 , 1353 , 2832 , 1340 , 1370 , 1430 , 1451 , 1454 , 1716 , 1439 , 1498 , 1516 , 2924 , 1585 , 1485 , 1549 , 1553 , 1573 , 1403 , 1532 , 1612 , 1610 , 1632 , 1330 , 1585 , 1675 , 1630 , 1687 , 1687 , 1656 , 1711 , 1730 , 1733 , 1463 , 1690 , 1770 , 1795 , 1790 , 1868 , 1764 , 1824 , 1820 , 1839 , 2174 , 1786 , 1863 , 1883 , 1882 , 1905 , 1856 , 1926 , 1898 , 1937 , 292 , 1712 , 1982 , 1985 , 2000 , 2075 , 1940 , 2023 , 2025 , 2044 , 2098 , 1759 , 2056 , 2080 , 2079 , 2098 , 2034 , 2103 , 2123 , 2118 , 2144 , 1846 , 2159 , 2123

In [14]:
while True:
    if p12dd != -1:
        print(p12dd)
        p12dd = -1

942633204
942633835
1470
1516
1467
1531
1531
1546
1554
1527
1569
1584
1615
1606
1590
1637
1644
1652
1667
1640
1690
1706
1714
1729
1687
1744
1744
1766
1790
1741
1804
1804
1827
1834
1793
1835
1856
1865
1881
1839
1895
1896
1918
1942
1900
1956
1971
1979
1979
1930
1995
1993
2018
2024
1991
2033
2055
2062
2077
2021
2093
2093
2116
2115
2074
2138
2138
2154
2153
2142
2175
2177
2184
2199
2164
2206
2214
2228
2238
2195
2251
2252
2260
2274
2211
2267
2296
2298
2297
2262
2305
2313
2314
2328
2295
2328
2358
2350
2351
2317
2350
2366
2365
2367
2326
2389
2396
2389
2388
2348
2381
2411
2405
2419
2370
2412
2411
2411
2435
2385
2427
2420
2426
2433
2385
2426
2427
2434
2434
2378
2442
2435
2434
2434
2371
2435
2434
2426
2420
2385
2434
2428
2434
2426
2385
2435
2434
2442
2434
2363
2412
2413
2404
2419
2369
2412
2389
2388
2389
2340
2395
2382
2365
2373
2324
2367
2357
2350
2359
2301
2351
2328
2328
2335
2271
2320
2312
2312
2291
2249
2297
2275
2275
2268
2197
2251
2253
2238
2236
2179
2222
2223
2215
2191
2149
2176
2176
2176


Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
KeyboardInterrupt: 


In [11]:
%sendtofile main.py
from machine import UART, Pin, time_pulse_us
import time, socket, urandom
from BNO055_class import BNO055
from UDPblackviewphone import connectActivePhone

fconfig = dict(x.strip().split(None, 1)  for x in open("config.txt"))
pinled = Pin(int(fconfig["pinled"]), Pin.OUT)
pinled.on()

pitchServo = Pin(int(fconfig["pitchServo"]), Pin.IN)
rollServo = Pin(int(fconfig["rollServo"]), Pin.IN)
while True:
    #print(pitchServo.value())
    #print(rollServo.value())
    #time.sleep(1)
    print(time_pulse_us(rollServo, 1))
    print(time.ticks_ms())


Sent 17 lines (547 bytes) to main.py.


In [76]:
#%sendtofile main.py
from machine import UART, Pin
import time, socket, urandom

fconfig = dict(x.strip().split(None, 1)  for x in open("config.txt"))

pServoUp = -1
pServoDown = -1
#rServoUp = -1
#rServoDown = -1

def pitchServoIRQ(p):
    global pServoUp, pServoDown
    t = time.ticks_us()
    if p.value() == 1:
        pServoUp = t
    else:
        pServoDown = t - pServoUp
        pServoUp = -1
        
def rollServoIRQ(p):
    global rServoUp, rServoDown
    if p.value() == 1:
        rServoUp = time.ticks_us()
    else:
        rServoDown = time.ticks_us() - rServoUp
        rServoUp = -1
        
pitchServo = Pin(int(fconfig["pitchServo"]), Pin.IN)
rollServo = Pin(int(fconfig["rollServo"]), Pin.IN)


pitchServo.irq(handler=pitchServoIRQ, trigger=Pin.IRQ_FALLING | Pin.IRQ_RISING)
rollServo.irq(handler=rollServoIRQ, trigger=Pin.IRQ_FALLING | Pin.IRQ_RISING)

while True:
    if pServoDown != -1:
        print(pServoDown,',',end=" ")
        pServoDown = -1
    if rServoDown != -1:
        print('Roll', rServoDown)
        rServoDown = -1

Roll 10334471
........55226965 , 55227462 , 55227921 , 915 , 920 , 940 , 959 , 927 , 958 , 948 , 977 , 994 , 994 , 1016 , 999 , 1038 , 1035 , 1034 , 1048 , 1042 , 1044 , 1073 , 1095 , 1092 , 1096 , 1114 , 1135 , 1135 , 1154 , 1139 , 1174 , 1172 , 1189 , 1160 , 1198 , 1207 , 1229 , 1233 , 1248 , 1240 , 1273 , 1286 , 1290 , 1311 , 1291 , 1362 , 1330 , 1350 , 1345 , 1348 , 1369 , 1388 , 1409 , 1406 , 1410 , 1427 , 1445 , 1446 , 1436 , 1435 , 1484 , 1500 , 1503 , 1525 , 1507 , 1544 , 1543 , 1562 , 1579 , 1569 , 1601 , 1575 , 1618 , 1619 , 1627 , 1643 , 1659 , 1679 , 1680 , 1681 , 1698 , 1719 , 1719 , 1734 , 1690 , 1758 , 1777 , 1777 , 1792 , 1781 , 1817 , 1813 , 1835 , 1838 , 1838 , 1857 , 1875 , 1842 , 1893 , 1879 , 1914 , 1911 , 1935 , 1937 , 1937 , 1973 , 1972 , 1973 , 1994 , 1993 , 2011 , 2012 , 2030 , 2026 , 2034 , 2048 , 2069 , 2071 , 2087 , 2073 , 2110 , 2112 , 2084 , 2132 , 2112 , 2163 , 2155 , 2167 , 2174 , 2149 , 2184 , 2188 , 2193 , 2208 , 2202 , 2201 , 2239 , 2241 , 2246 , 2234

Traceback (most recent call last):
  File "<stdin>", line 42, in <module>
KeyboardInterrupt: 
